In [2]:
import pandas as pd
import pickle

import os
import string
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import PorterStemmer

In [3]:
train = pd.read_csv('train_dist.csv')
test = pd.read_csv('test_dist.csv')

In [4]:
train.columns

Index(['CandidateID', 'Designation',
       'Total no of years Experience [before joining Piramal]',
       'Previous Industry worked with [before joining Piramal]',
       'Name of your Previous Organization / Company',
       'How many Organization that you have worked before joining Piramal Finance ?',
       'Average Incentive [per month] earned in your pervious company ?',
       'How did you come to know about the role at Piramal Finance ?',
       'Which Products you are selling in your pervious role ?',
       'What was the average ticket size handled at your end in previous role ?',
       'How many members are there in your family ?',
       'How many are earning family members ? [Other then yourself]2',
       'How many members are dependent on you ?', 'Department', 'DOJ',
       'Location Code', 'Residential Pincode', 'Branch Pincode', 'Performance',
       'education', 'train_vector', 'branch_lat', 'branch_lon',
       'residential_lat', 'residential_lon', 'distance'],
   

In [5]:
train = train.drop(['branch_lat','branch_lon', 'residential_lat', 'residential_lon'], axis=1)
test = test.drop(['branch_lat','branch_lon', 'residential_lat', 'residential_lon'], axis=1)

In [6]:
with open("resume_data.pkl", "rb") as file:
    resume_text = pickle.load(file)
    
with open("resume_data_test.pkl", "rb") as file:
    resume_text_test = pickle.load(file)

In [8]:
def process_string(text):

    final_string = ""

    # Convert the text to lowercase
    text = text.lower()

    # Remove punctionation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Remove stop words and usless words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    text_filtered = [word for word in text if not word in useless_words]

    # Stem the text with NLTK PorterStemmer
    stemmer = PorterStemmer() 
    text_stemmed = [stemmer.stem(y) for y in text_filtered]

    # Join the words back into a string
    final_string = ' '.join(text_stemmed)

    return final_string

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91939\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [12]:
import os
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tqdm import tqdm

train_tagged = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(resume_text)]
test_tagged = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(resume_text_test)]

# Train the Doc2Vec model
model = Doc2Vec(vector_size=700, min_count=2, epochs=40)
model.build_vocab(train_tagged)
model.train(train_tagged, total_examples=model.corpus_count, epochs=model.epochs)

# Get the document vectors
train_vectors = [model.infer_vector(doc.words) for doc in tqdm(train_tagged)]
test_vectors = [model.infer_vector(doc.words) for doc in tqdm(test_tagged)]

100%|████████████████████████████████████████████████████████████████████████████████| 187/187 [00:06<00:00, 30.49it/s]


In [7]:
# train['vectors'] = train_vectors
# test['vectors'] = test_vectors
train = train.drop(['train_vector', 'CandidateID', 'Designation', 'Location Code', 'Residential Pincode', 'Branch Pincode'], axis=1)
test = test.drop(['train_vector', 'CandidateID', 'Designation', 'Location Code', 'Residential Pincode', 'Branch Pincode'], axis=1)

In [8]:
train['resume'] = pd.DataFrame(resume_text, columns=['resume'])
test['resume'] = pd.DataFrame(resume_text_test, columns=['resume'])

In [9]:
train.to_csv('train1.csv', index=False)
test.to_csv('test1.csv', index=False)

In [13]:
from autogluon.core.dataset import TabularDataset
train_data = TabularDataset('train.csv')
train_data.head(30)
train_data.columns

Loaded data from: train.csv | Columns = 21 / 21 | Rows = 745 -> 745


Index(['CandidateID', 'Designation', 'Have you Completed your Graduation ?',
       'Highest Educational Qualification',
       'Total no of years Experience [before joining Piramal]',
       'Previous Industry worked with [before joining Piramal]',
       'Name of your Previous Organization / Company',
       'How many Organization that you have worked before joining Piramal Finance ?',
       'Average Incentive [per month] earned in your pervious company ?',
       'How did you come to know about the role at Piramal Finance ?',
       'Which Products you are selling in your pervious role ?',
       'What was the average ticket size handled at your end in previous role ?',
       'How many members are there in your family ?',
       'How many are earning family members ? [Other then yourself]2',
       'How many members are dependent on you ?', 'Department', 'DOJ',
       'Location Code', 'Residential Pincode', 'Branch Pincode',
       'Performance'],
      dtype='object')

In [14]:
from autogluon.tabular import TabularPredictor 
time_limit = 90
predictor = TabularPredictor(label='Performance',eval_metric='f1').fit(train_data)

No path specified. Models will be saved in: "AutogluonModels\ag-20240411_133137"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240411_133137"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          16
Memory Avail:     

In [18]:
test2 = pd.read_csv('test.csv',  encoding='ISO-8859-1')
test2.to_csv('test2.csv', index=False)

In [19]:
test_data = TabularDataset('test2.csv')
label = 'CandidateID'
y_pred = predictor.predict((test_data))
y_pred.head()

Loaded data from: test2.csv | Columns = 20 / 20 | Rows = 187 -> 187


0    1
1    1
2    0
3    0
4    0
Name: Performance, dtype: int64

In [20]:
sub=pd.DataFrame()
sub['CandidateID'],sub['Performance']=test_data['CandidateID'],y_pred
sub.to_csv('submission_7.csv', index=False)
sub

,CandidateID,Performance
0,EMP0521,1
1,EMP0613,1
2,EMP0136,0
3,EMP0351,0
4,EMP0049,0
...,...,...
182,EMP0401,1
183,EMP0408,0
184,EMP0248,0
185,EMP0148,0
